In [272]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor,  BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
import xgboost
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score


from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import altair as alt
import datetime as dt

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [89]:
def read_data(stock='AAPL', period = "60d", interval = "15m", intra = True):
    '''
    reading the data
    '''

    df = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,)


    return df

In [175]:
def read_data_enhanced(stock='AAPL', period = "60d", interval = "15m", intra = True):
    '''
    reading the data
    '''
    
    if intra ==True:


        dfx = pdr.get_data_yahoo(stock,period = period,

                # fetch data by interval (including intraday if period < 60 days)
                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = interval,).reset_index()



        df_nasaq = read_data(stock='NDAQ', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]

        df_nasaq['fn'] = ((df_nasaq['Open'].shift(1)/df_nasaq['Adj Close'].shift(1))-1)
        df_nasaq= df_nasaq[['Datetime','fn']]


        df_sp = read_data(stock='^GSPC', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]
        df_sp['fs'] = ((df_sp['Open'].shift(1)/df_sp['Adj Close'].shift(1))-1)
        df_sp= df_sp[['Datetime','fs']]


        dfx = pd.merge(dfx,df_nasaq,on = ['Datetime'], how = 'left')

        dfx = pd.merge(dfx,df_sp,on = ['Datetime'], how = 'left')
        
    elif intra ==False:
        dfx = pdr.get_data_yahoo(stock,period = period,

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = interval,).reset_index()



        df_nasaq = read_data(stock='NDAQ', period = period, interval = interval).reset_index()[['Date','Open','Adj Close']]

        df_nasaq['fn'] = ((df_nasaq['Open'].shift(1)/df_nasaq['Adj Close'].shift(1))-1)
        df_nasaq= df_nasaq[['Date','fn']]


        df_sp = read_data(stock='^GSPC', period = period, interval = interval).reset_index()[['Date','Open','Adj Close']]
        df_sp['fs'] = ((df_sp['Open'].shift(1)/df_sp['Adj Close'].shift(1))-1)
        df_sp= df_sp[['Date','fs']]


        dfx = pd.merge(dfx,df_nasaq,on = ['Date'], how = 'left')

        dfx = pd.merge(dfx,df_sp,on = ['Date'], how = 'left')
        


        
    return dfx

In [5]:
df1 = read_data(stock='AAPL').reset_index()

[*********************100%***********************]  1 of 1 completed


In [6]:
df2 = df1.copy()

In [7]:
# here is the tail of the df which has the most recent data
df1.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2020-10-12 09:30:00-04:00,120.059998,120.150002,119.284500,119.690002,119.690002,11041255
1,2020-10-12 09:32:00-04:00,119.699997,120.150002,119.400002,120.000000,120.000000,2273540
2,2020-10-12 09:34:00-04:00,120.000000,120.680000,119.910004,120.559998,120.559998,2563680
3,2020-10-12 09:36:00-04:00,120.750000,120.910004,120.750000,120.906601,120.906601,2064244
4,2020-10-12 09:38:00-04:00,121.000000,121.000000,120.581596,120.762299,120.762299,2285976


In [8]:
df2 = read_data_enhanced(stock='AAPL')
df2.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,fn,fs
0,2020-10-12 09:30:00-04:00,120.059998,120.150002,119.284500,119.690002,119.690002,11041255,NaN,NaN
1,2020-10-12 09:32:00-04:00,119.699997,120.150002,119.400002,120.000000,120.000000,2273540,-0.000079,-0.000149
2,2020-10-12 09:34:00-04:00,120.000000,120.680000,119.910004,120.559998,120.559998,2563680,0.001988,-0.000263
3,2020-10-12 09:36:00-04:00,120.750000,120.910004,120.750000,120.906601,120.906601,2064244,-0.002065,-0.000822
4,2020-10-12 09:38:00-04:00,121.000000,121.000000,120.581596,120.762299,120.762299,2285976,0.000040,-0.000585


### Models

In [9]:
# We have the follwoing model already established

In [9]:
def test_stock(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    df = dfx.copy()
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))


    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')

    

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [10]:
# now let's see the power of feature engineering:
#we will add the daily return as a feature for this model

In [11]:
def test_stock_w_feature(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    #print(len(df))


    df = dfx.copy()
    
    # for now the following do not add any predictiv power
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))
    

        


    
    df['f01'] = ((df['Open']/df['Close'].shift(1))-1)*100
    df['O-1'] = df['Open'].shift(1)
    df['O-2'] = df['Open'].shift(2)
    df['O-3'] = df['Open'].shift(3)
    df['O-4'] = df['Open'].shift(4)
    df['O-5'] = df['Open'].shift(5)
    
    df['c-1'] = df['Close'].shift(1)
    df['c-2'] = df['Close'].shift(2)
    df['c-3'] = df['Close'].shift(3)
    df['c-4'] = df['Close'].shift(4)
    df['c-5'] = df['Close'].shift(5)
    
    
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    




### Performance Comparision

In [12]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df1, intra = True)

6303   2020-11-25 11:38:34-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.998
error 0.14073852965767322
old price =  116.11730194091797
predicted =  116.14334753121437
difference =  0.026045590296405408


In [13]:
test_stock_w_feature(df1, intra = True)

model score: 0.998
error 0.13902068205813273
old price =  116.11730194091797
predicted =  116.14578795972683
difference =  0.028486018808862923


---

With NASDAQ dataframe :

In [14]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df2, intra = True)

6303   2020-11-25 11:38:34-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.998
error 0.14052909034976063
old price =  116.11730194091797
predicted =  116.2861032081868
difference =  0.16880126726883304


In [15]:
test_stock_w_feature(df2, intra = True)

model score: 0.998
error 0.13911352723207823
old price =  116.11730194091797
predicted =  116.27391523913255
difference =  0.15661329821458025


In [49]:
# Do we same the same gain if use a different modeel: Random Forest ?

In [16]:
def test_stock_grad(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """

    df = dfx.copy()
    df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    



    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False, random_state = 2020)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor(random_state = 2020)
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
        #search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    error = np.sqrt(mean_squared_error(y_pred,y_test))
    print('error',error)
#     X_train = X[:-1]
#     y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    y_test = search.predict(X_test)[0]
    diff = y_test - old
        
    if diff > error :
        print('winner')
        print('old price = ',old)
        print('predicted = ',y_test)
        print('difference = ', diff)





In [222]:
def test_stock_grad_feature(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """


    df = dfx.copy()
    df['M7'] =df['Adj Close'].rolling(window=7).mean()
    df['M7d'] =df['Adj Close'].rolling(window=7).std()
    df['M14'] =df['Adj Close'].rolling(window=14).mean()
    df['M14d'] =df['Adj Close'].rolling(window=14).std()
    df['M21'] =df['Adj Close'].rolling(window=21).mean()
    df['M21d'] =df['Adj Close'].rolling(window=21).std()


    df['f01'] = ((df['Open'].shift(1)/df['Adj Close'].shift(1))-1)
    df['f02'] = ((df['Open'].shift(2)/df['Adj Close'].shift(2))-1)
    df['f03'] = ((df['Open'].shift(3)/df['Adj Close'].shift(3))-1)
    df['v01'] = ((df['Volume'].shift(1)/df['Volume'].shift(1))-1)
#     df['f02'] = ((df['Open'].shift(1)/df['Adj Close'].shift(2))-1)*100
#     df['O-1'] = df['Open'].shift(1)
#     df['O-2'] = df['Open'].shift(2)
#     df['O-3'] = df['Open'].shift(3)
#     df['O-4'] = df['Open'].shift(4)
#     df['O-5'] = df['Open'].shift(5)
    
#     df['c-1'] = df['Close'].shift(1)
#     df['c-2'] = df['Close'].shift(2)
#     df['c-3'] = df['Close'].shift(3)
#     df['c-4'] = df['Close'].shift(4)
#     df['c-5'] = df['Close'].shift(5)


    
    


    #print(len(df))

    
    if intra ==True:
            print(df.tail(1)['Datetime'])
            old = df.tail(1).loc[:,'Open'].to_numpy()[0]
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            
            
            df['day_of_the_week'] = df['Datetime'].dt.dayofweek
            df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            print(df.tail(1)['Date'])
            old = df.tail(1).loc[:,'Open'].to_numpy()[0]
            df['Date'] = pd.to_datetime(df['Date'])
            
            df['day_of_the_week'] = df['Date'].dt.dayofweek
            df['day_of_month'] = df['Date'].apply(lambda x:int(str(x)[8:10]))
            df['Date']=df['Date'].map(dt.datetime.toordinal)
            
            df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False, random_state = 2020)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper','status_sma_lower','status_sma_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_mode = RandomForestRegressor(random_state = 2020, n_estimators = 2, n_jobs = 24)
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipelin = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_mode)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
#     param_grid ={'Regressor__bootstrap': [False], 'Regressor__max_depth': [10,20,50],
#                  'Regressor__min_samples_leaf': [1,4,6], 'Regressor__min_samples_split': [5,10],
#                  'Regressor__n_estimators': [10,50]}
    
    param_grid ={'Regressor__n_estimators': [200],
                 'Regressor__max_depth': [15]
                
                }
    
    
    search = GridSearchCV(reg_pipelin,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    print(search.best_params_)
    print("model training score: %.3f" % search.score(X_train, y_train))
    print("model test score: %.3f" % search.score(X_test, y_test))
    
    # training the model on the whole data set

    #search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    error = np.sqrt(mean_squared_error(y_pred,y_test))
    print('error',error)
#     X_train = X[:-1]
#     y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    y_test = search.predict(X_test)[0]
    diff = y_test - old
        
    if diff > error :
        print('winner')
        print('old price = ',old)
        print('predicted = ',y_test)
        print('difference = ', diff)
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    
    return error

    


### Creating Dataframes

In [97]:
df_2 = read_data(stock='AAPL',interval = "2m").reset_index()

[*********************100%***********************]  1 of 1 completed


In [99]:
df_5 = read_data(stock='AAPL',interval = "5m").reset_index()

[*********************100%***********************]  1 of 1 completed


In [100]:
df_15 = read_data(stock='AAPL',interval = "15m").reset_index()

[*********************100%***********************]  1 of 1 completed


In [101]:
df_30 = read_data(stock='AAPL',interval = "30m").reset_index()

[*********************100%***********************]  1 of 1 completed


In [109]:
df_60 = read_data(stock='AAPL',interval = "60m").reset_index()

[*********************100%***********************]  1 of 1 completed


In [180]:
df_360 = read_data(stock='AAPL',period = "200d",interval = "1d").reset_index()

[*********************100%***********************]  1 of 1 completed


In [98]:
dfe_2 = read_data_enhanced(stock='AAPL',interval = "2m")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### let's conduct a sensitivity study for the effect of interval

### Models

In [179]:
# let's predict he clost price for TSLA after 2 minutes
test_stock_grad_feature(df_2,intra = True)

6336   2020-11-25 12:43:08-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 1.000
model test score: 0.998
error 0.10014484025552677
old price =  115.96499633789062
predicted =  116.00634801228841
difference =  0.041351674397787974


In [160]:
test_stock_grad_feature(df_5,intra = True)

4641   2020-11-25 12:43:21-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.999
model test score: 0.985
error 0.22299104049349214
old price =  115.94499969482422
predicted =  115.9198316040039
difference =  -0.02516809082031557


In [161]:
test_stock_grad_feature(df_15,intra = True)

1547   2020-11-25 12:43:32-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.998
model test score: 0.943
error 0.43089306398385796
old price =  115.9574966430664
predicted =  116.10355144924586
difference =  0.14605480617944977


In [162]:
test_stock_grad_feature(df_30,intra = True)

773   2020-11-25 12:30:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.997
model test score: 0.909
error 0.5553211694495666
old price =  115.94000244140625
predicted =  115.9382512919108
difference =  -0.0017511494954476348


In [163]:
test_stock_grad_feature(df_60,intra = True)

417   2020-11-25 12:45:45-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.995
model test score: 0.864
error 0.7100651564487155
old price =  116.02999877929688
predicted =  116.07517801920572
difference =  0.045179239908847535


In [181]:
test_stock_grad_feature(df_360,intra = False)

199   2020-11-25
Name: Date, dtype: datetime64[ns]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.992
model test score: 0.654
error 1.891168957823174
old price =  115.55000305175781
predicted =  114.87022313435872
difference =  -0.6797799173990882


> As we increases the time interval, we have less data and hence less accuracy
> Once we went beyond the intra to full day, then we have error > 1 and sometimes the score is negative deponding on how much data we included, it is very senstive in this regard. Not stable

### let's conduct a sensitivity study for the effect of enhanced data frame with other stocks 

In [177]:
dfe_2 = read_data_enhanced(stock='AAPL',period = "60d",interval = "2m", intra = True).reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


>  Using the df2 improved th result by 0.1

In [178]:
test_stock_grad_feature(dfe_2,intra = True)

6354   2020-11-25 13:20:07-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 1.000
model test score: 0.998
error 0.10382104098005492
old price =  116.14929962158203
predicted =  116.24587849934896
difference =  0.0965788777669303


> reducing the n_estimators seems to increase overfitting

> df2 did not improve the accuracy for the 2 minutes.

----

In [185]:
dfe_360 = read_data_enhanced(stock='AAPL',period = "200d",interval = "1d", intra = False).reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [186]:
test_stock_grad_feature(dfe_360,intra = False)

199   2020-11-25
Name: Date, dtype: datetime64[ns]
{'Regressor__max_depth': 20, 'Regressor__n_estimators': 150}
model training score: 0.992
model test score: 0.672
error 1.8416963107101247
old price =  115.55000305175781
predicted =  114.73751286824545
difference =  -0.8124901835123666


> Here we see some improvements for the daily predictions

In [187]:
## Let's trade. 2 minutes

In [196]:
# stocks = ['AAL', 'BAC','AMD','PLUG','WORK']

# for stock in stocks :
#     df_2 = read_data(stock=stock,interval = "2m").reset_index()
#     error = test_stock_grad_feature(df_2,intra = True)
#     print(stock,error)
    

In [235]:
df_2 = read_data(stock='AMD',interval = "5m").reset_index()


[*********************100%***********************]  1 of 1 completed


In [236]:
test_stock_grad_feature(df_2,intra = True)

4664   2020-11-25 14:39:49-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 15, 'Regressor__n_estimators': 200}
model training score: 0.999
model test score: 0.990
error 0.22073264691875563
old price =  86.9800033569336
predicted =  87.08065473556519
difference =  0.10065137863159634


0.22073264691875563

In [237]:
## Transforming this into a classification Taks

In [256]:
df_trial = read_data(stock='AMD',interval = "5m").reset_index()



[*********************100%***********************]  1 of 1 completed


In [315]:
def test_stock_grad_classify(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """


    df = dfx.copy()
    df['tomorrow_closing'] = df['Adj Close'].shift(-1)
    df = df.dropna()
    df['go_up'] = np.where(df['tomorrow_closing'] > df['Adj Close'],1,0)
    
    df['M7'] =df['Adj Close'].rolling(window=7).mean()
    df['M7d'] =df['Adj Close'].rolling(window=7).std()
    df['M14'] =df['Adj Close'].rolling(window=14).mean()
    df['M14d'] =df['Adj Close'].rolling(window=14).std()
    df['M21'] =df['Adj Close'].rolling(window=21).mean()
    df['M21d'] =df['Adj Close'].rolling(window=21).std()


    df['f01'] = ((df['Open'].shift(1)/df['Adj Close'].shift(1))-1)
    df['f02'] = ((df['Open'].shift(2)/df['Adj Close'].shift(2))-1)
    df['f03'] = ((df['Open'].shift(3)/df['Adj Close'].shift(3))-1)
    df['v01'] = ((df['Volume'].shift(1)/df['Volume'].shift(1))-1)
#     df['f02'] = ((df['Open'].shift(1)/df['Adj Close'].shift(2))-1)*100
#     df['O-1'] = df['Open'].shift(1)
#     df['O-2'] = df['Open'].shift(2)
#     df['O-3'] = df['Open'].shift(3)
#     df['O-4'] = df['Open'].shift(4)
#     df['O-5'] = df['Open'].shift(5)
    
#     df['c-1'] = df['Close'].shift(1)
#     df['c-2'] = df['Close'].shift(2)
#     df['c-3'] = df['Close'].shift(3)
#     df['c-4'] = df['Close'].shift(4)
#     df['c-5'] = df['Close'].shift(5)


    
    


    #print(len(df))

    
    if intra ==True:
            print(df.tail(1)['Datetime'])
            old = df.tail(1).loc[:,'Open'].to_numpy()[0]
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            
            
            df['day_of_the_week'] = df['Datetime'].dt.dayofweek
            df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            
    
    elif intra ==False:
            print(df.tail(1)['Date'])
            old = df.tail(1).loc[:,'Open'].to_numpy()[0]
            df['Date'] = pd.to_datetime(df['Date'])
            
            df['day_of_the_week'] = df['Date'].dt.dayofweek
            df['day_of_month'] = df['Date'].apply(lambda x:int(str(x)[8:10]))
            df['Date']=df['Date'].map(dt.datetime.toordinal)
            
            #df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()


     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()

    df = df.drop(columns = ['Close','tomorrow_closing'])
    y = df['go_up']
    X = df.drop(columns =['go_up','Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False, random_state = 2020)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_mode = RandomForestClassifier(random_state = 2020, n_estimators = 2, n_jobs = 24)
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipelin = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_mode)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
#     param_grid ={'Regressor__bootstrap': [False], 'Regressor__max_depth': [10,20,50],
#                  'Regressor__min_samples_leaf': [1,4,6], 'Regressor__min_samples_split': [5,10],
#                  'Regressor__n_estimators': [10,50]}
    
    param_grid ={'Regressor__n_estimators': [200],
                 'Regressor__max_depth': [15]
                
                }
    
    
    search = GridSearchCV(reg_pipelin,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    print(search.best_params_)
    print("model training score: %.3f" % search.score(X_train, y_train))
    print("model test score: %.3f" % search.score(X_test, y_test))
    
    # training the model on the whole data set

    #search.fit(X_train,y_train)
    y_pred = search.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print('accuracy = ', accuracy_score(y_test,y_pred),'precision = ', precision_score(y_test,y_pred),'recall = ', recall_score(y_test,y_pred))
    
#     error = np.sqrt(mean_squared_error(y_pred,y_test))
#     print('error',error)
# #     X_train = X[:-1]
# #     y_train = y[:-1]
#     X_test = X[-1:]
#     y_test = y[-1:]
#     y_test = search.predict(X_test)[0]
#     diff = y_test - old
        
#     if diff > error :
#         print('winner')
#         print('old price = ',old)
#         print('predicted = ',y_test)
#         print('difference = ', diff)
#     print('old price = ',old)
#     print('predicted = ',y_test)
#     print('difference = ', diff)
    
    #return error

    



In [ ]:
# short term

In [280]:
df_2 = read_data(stock='AMD',interval = "2m").reset_index()
test_stock_grad_classify(df_2,intra = True)


[*********************100%***********************]  1 of 1 completed
6414   2020-11-25 15:20:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
{'Regressor__max_depth': 15, 'Regressor__n_estimators': 200}
model training score: 0.996
model test score: 0.499
[[614 171]
 [626 181]]
accuracy =  0.4993718592964824 precision =  0.5142045454545454 recall =  0.22428748451053285


In [ ]:
# long term

In [316]:
df_360 = read_data(stock='AAPL',period = "366d",interval = "1d").reset_index()
test_stock_grad_classify(df_360,intra = False)

[*********************100%***********************]  1 of 1 completed
364   2020-11-24
Name: Date, dtype: datetime64[ns]
{'Regressor__max_depth': 15, 'Regressor__n_estimators': 200}
model training score: 1.000
model test score: 0.471
[[38  0]
 [45  2]]
accuracy =  0.47058823529411764 precision =  1.0 recall =  0.0425531914893617


In [ ]:
# tn =38
# tp = 2

# fp = 0
# fn =1 



In [317]:
# for some reason when I include the 50 sma , I got most of the postives there, high recall, but with low precision